<a href="https://colab.research.google.com/github/xivh/stylegan2-colab-notebooks/blob/master/Convert_images_to_shards.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 1.x

# Get the converter

In [0]:
!git clone https://github.com/shawwn/danbooru-tools

Cloning into 'danbooru-tools'...
remote: Enumerating objects: 174, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 174 (delta 56), reused 173 (delta 55), pack-reused 0
Receiving objects: 100% (174/174), 35.61 KiB | 5.09 MiB/s, done.
Resolving deltas: 100% (56/56), done.


# Get your images



For example, get your zipped, processed images from Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Change the first argument to the path of your zip file - you can find it in the side browser and right click to copy the path

In [0]:
!cp "/content/drive/My Drive/Posters/sergeposterlad.zip" /content/

If your file was zipped, you need to unzip it

In [0]:
!mkdir images/
!unzip sergeposterlad.zip -d images/

Archive:  sergeposterlad.zip
  inflating: images/1525490284611517607.jpg  
  inflating: images/1525493974928060020.jpg  
  inflating: images/1525497771192701711.jpg  
  inflating: images/1526024798064685921.jpg  
  inflating: images/1526669030173478391.jpg  
  inflating: images/1527475481779444000.jpg  
  inflating: images/1528339479689359316.jpg  
  inflating: images/1528877061468735574.jpg  
  inflating: images/1530409857467142159.jpg  
  inflating: images/1531166616662206297.jpg  
  inflating: images/1531931500232456777.jpg  
  inflating: images/1532677962235061171.jpg  
  inflating: images/1533278866210402182.jpg  
  inflating: images/1534124685167722674.jpg  
  inflating: images/1534890511550513289.jpg  
  inflating: images/1535547627017846252.jpg  
  inflating: images/1536238876770962626.jpg  
  inflating: images/1537042230212432114.jpg  
  inflating: images/1542861668459265505.jpg  
  inflating: images/1543392767354703709.jpg  
  inflating: images/1544175753071318968.jpg  
  inf

You want your images to be in images/\*.jpg, not images/another_folder/\*.jpg

# Convert your images to shards

Shards are used to shuffle the images. The minimum is 8 (8 tpu cores), the default is 2048. I have set it to 128 below, but you can change it if you want. You may get fewer shards than you set because each shard must have the same number of images.

Is too few images/shard a problem? check

In [0]:
!find images/ -type f -name '*.jpg' > images.txt
!python danbooru-tools/danbooru_to_tfrecord.py --name images --out 'sergeposterlad-512-10' --files images.txt --shards 10

INFO:tensorflow:training records to ensure we are distributing classes across the batches.
I0614 03:21:33.899071 139908308895616 danbooru_to_tfrecord.py:448] training records to ensure we are distributing classes across the batches.
INFO:tensorflow:Glob all the training files.
I0614 03:21:33.899447 139908308895616 danbooru_to_tfrecord.py:456] Glob all the training files.
INFO:tensorflow:Processing the training data.
I0614 03:21:33.901538 139908308895616 danbooru_to_tfrecord.py:474] Processing the training data.
2020-06-14 03:21:33.966142: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-14 03:21:33.966416: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-14 03:21:33.967862: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-14 03:21:33.972909: I tensorflow/stream_executor/pla

Upload to Google bucket


1.   Authenticate yourself. This one might not work... 
2.   Upload your files



In [0]:
!gcloud auth login
!gcloud config set project cybernetic-song-276823 

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&code_challenge=sLCC5Vv0pPe-5qI-mrPTrWu0lA7XSkK9tO_0u5cZtWM&code_challenge_method=S256&access_type=offline&response_type=code&prompt=select_account


Enter verification code: 4/0wETDUD7jSfMuCfpDrw-MuRV_mLHIBFUz0u-O9RVZ7QxgfTKFiNa2FQ

You are now logged in as [jt6fz@virginia.edu].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID
Updated property [core/project].


In [0]:
!gsutil -m cp sergeposterlad-512-10/* gs://full-of-posters-euw4a/datasets/sergeposterlad-512-10

Copying file://sergeposterlad-512-10/images-00004-of-00010 [Content-Type=application/octet-stream]...
Copying file://sergeposterlad-512-10/images-00005-of-00010 [Content-Type=application/octet-stream]...
Copying file://sergeposterlad-512-10/images-00000-of-00010 [Content-Type=application/octet-stream]...
Copying file://sergeposterlad-512-10/images-00002-of-00010 [Content-Type=application/octet-stream]...
Copying file://sergeposterlad-512-10/images-00006-of-00010 [Content-Type=application/octet-stream]...
Copying file://sergeposterlad-512-10/images-00001-of-00010 [Content-Type=application/octet-stream]...
Copying file://sergeposterlad-512-10/images-00009-of-00010 [Content-Type=application/octet-stream]...
Copying file://sergeposterlad-512-10/images-00008-of-00010 [Content-Type=application/octet-stream]...
Copying file://sergeposterlad-512-10/images-00007-of-00010 [Content-Type=application/octet-stream]...
Copying file://sergeposterlad-512-10/images-00003-of-00010 [Content-Type=applicati

# Extra: make the necessary tfrecords

Currently, you need to make a dummy folder with empty tfrecords

In [0]:
dataset = "sergeposterlad" # set this to your dataset name

from pathlib import Path

Path("dummy_data", dataset).mkdir(parents=True, exist_ok=True)

for i in range(2, 11):
  Path("dummy_data", dataset, dataset + "-r%02d.tfrecords" % i).touch()

Assuming you are already authenticated (see previous sections), upload

In [0]:
!gsutil -m cp -r dummy_data/ gs://full-of-posters-euw4a/dummy_data

Copying file://dummy_data/sergeposterlad/sergeposterlad-r10.tfrecords [Content-Type=application/octet-stream]...
Copying file://dummy_data/sergeposterlad/sergeposterlad-r04.tfrecords [Content-Type=application/octet-stream]...
Copying file://dummy_data/sergeposterlad/sergeposterlad-r03.tfrecords [Content-Type=application/octet-stream]...
Copying file://dummy_data/sergeposterlad/sergeposterlad-r08.tfrecords [Content-Type=application/octet-stream]...
Copying file://dummy_data/sergeposterlad/sergeposterlad-r06.tfrecords [Content-Type=application/octet-stream]...
Copying file://dummy_data/sergeposterlad/sergeposterlad-r09.tfrecords [Content-Type=application/octet-stream]...
Copying file://dummy_data/sergeposterlad/sergeposterlad-r05.tfrecords [Content-Type=application/octet-stream]...
Copying file://dummy_data/sergeposterlad/sergeposterlad-r07.tfrecords [Content-Type=application/octet-stream]...
Copying file://dummy_data/sergeposterlad/sergeposterlad-r02.tfrecords [Content-Type=application/

# Extra: Quickly process your images

Only do this for a quick check - you should have already processed your images before using this notebook

Your original images will not be preserved, so if you mess up, you'll have to download them again (or you could make a copy of the folder)

In [0]:
!pip install Augmentor
!pip install fastai

Check the number of images

In [0]:
!ls images | wc -l

1417


Resize all your images to the right dimensions



In [0]:
size = 512 # set this - I am making squares

import Augmentor
p = Augmentor.Pipeline("images")
p.resize(probability=1, height=size, width=size, resample_filter="ANTIALIAS")
p.process()

Processing <PIL.Image.Image image mode=RGB size=512x512 at 0x7FB47FFAF518>:   0%|          | 1/1416 [00:00<04:37,  5.11 Samples/s]

Initialised with 1416 image(s) found.
Output directory set to images/output.

Processing <PIL.Image.Image image mode=RGB size=512x512 at 0x7FB47C586CF8>: 100%|██████████| 1416/1416 [00:46<00:00, 30.13 Samples/s]


Move the output images back up to the images folder and remove the original images

In [0]:
!rm images/*.jpg
!mv images/output/* images/
!rm images/output -d

Check the images to make sure they all open

In [0]:
from fastai.vision import *
verify_images("images", delete=True)

Check how many images you have left - hopefully about the same, definitely not more

In [0]:
!ls images | wc -l

1416
